In [1]:
import requests
import json
import pandas as pd
import numpy as np
import pymysql
import os

In [2]:
# 1. 영화id를 사용하기 위해 영화정보 파일가져오기 
tmdb_movie_list = pd.read_csv('../data-files/total_tmdbmovielist.csv') 

C:\Users\bonomania\.conda\envs\ykseo\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,4,7,12,17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# 2. 영화정보 중 id만 가져오기
tmdb_movie_list.info()
tmdb_movie_id_list = tmdb_movie_list[["id"]]
# del tmdb_movie_list

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630186 entries, 0 to 630185
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             630186 non-null  int64  
 1   adult                  630158 non-null  object 
 2   backdrop_path          150102 non-null  object 
 3   belongs_to_collection  11995 non-null   object 
 4   budget                 630157 non-null  object 
 5   genres                 630155 non-null  object 
 6   homepage               69801 non-null   object 
 7   id                     630156 non-null  object 
 8   imdb_id                396414 non-null  object 
 9   original_language      630157 non-null  object 
 10  original_title         630130 non-null  object 
 11  overview               25238 non-null   object 
 12  popularity             630119 non-null  object 
 13  poster_path            426255 non-null  object 
 14  production_companies   630119 non-nu

In [4]:
print(tmdb_movie_id_list.shape)
tmdb_movie_id_list.head()

(630186, 1)


,id
0,2.0
1,3.0
2,5.0
3,6.0
4,8.0


In [40]:
## 영화 cast와 crew정보 3)영화id를 이용하여 cast와 crew정보 crawling  (너무 오래걸림)

all_crew_list = []
all_cast_list = []
error_list = []
for idx,  movie_id in enumerate(tmdb_movie_id_list["id"].values):
    try :
        cast_url="https://api.themoviedb.org/3/movie/{0}?api_key=36927ad1d2817ff0fa31947e47d186fd&append_to_response=credits"
        response = requests.get(cast_url.format(movie_id))
        json_object = response.content
        movie_data = json.loads(json_object)
        # cast_list += movie_data["credit"]["cast"]
        cast_list = movie_data["credits"]["cast"]
        for cast in cast_list:
            cast["movie_id"] = movie_id
            all_cast_list.append(cast)
        # crew_list += movie_data["credit"]['crew']
        crew_list = movie_data["credits"]["crew"]
        for crew in crew_list:
            crew["movie_id"] = movie_id
            all_crew_list.append(crew)
    except:
        error_list.append(movie_id)

In [ ]:
# cast 정보 csv파일로 저장 
casts=pd.DataFrame(all_cast_list)
casts.to_csv("crews.csv", mode='w')

In [ ]:
# cast정보 DB(mysql)에 저장 (테이블 만들어야 함)

conn = pymysql.connect(host="localhost",
                       database="finalteam3",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

cursor.execute('DELETE FROM ')
with conn.cursor() as cursor: # with가 종료될 때 cursor.close() 자동 호출
    cursor.execute()
    cursor.execute()
    conn.commit() # 이전에 실행된 SQL 결과를 확정

conn.close()